$\mathbf{1.}$ [R] Write a mathematical model to solve the assignment problem explained above. Define all
the variables and constraints clearly. Use appropriate notations and define appropriate sets
to be used in your optimization problem

$\mathbf{Solution:}Minimize\sum_{i=1,j=1}^{}\sum_{k=1,l=1}^{}c_{ik}d_{jl}y_{ijkl}$     

where $y_{ijkl}=x_{ij}x_{kl}$ 


s.t.
$\ \sum_{i}^{}y_{ijkl}=x_{kl}$        j,k,l=1,...n,

$\ \sum_{j}^{}y_{ikjl}=x_{kl}$        i,k,l=1....n

$\ y_{ikil}=y_{klij}$    i,j,k,l=1,2,.....n

$\ y_{ikil}>=0$          i,j,k,l=1....n

$C_{kl}$ denote the cost ith location to kth location 

$d_{jl}$ denote the demand jth location to lth loction


In [28]:
!pip install -q pyomo
from pyomo.environ import *

In [29]:
import numpy as np
import pandas as pd

In [30]:
data_txtfile1=np.loadtxt('lab7_ex1_c.txt',delimiter =',')
data_txtfile2=np.loadtxt('lab7_ex1_q.txt',delimiter =',')


In [31]:
print(data_txtfile1)


[[ 0. 11. 13.  9. 19. 20. 27. 13. 19. 11. 19. 10.]
 [11.  0.  8.  9. 11. 19. 10. 15. 12. 23. 24. 11.]
 [13.  8.  0. 18. 19. 19. 27. 27. 19. 24. 12. 17.]
 [ 9.  9. 18.  0. 19. 20. 10. 20. 21. 20. 27. 10.]
 [19. 11. 25. 19.  0. 21. 17. 26. 20. 14. 24. 17.]
 [20. 13. 20. 20. 21.  0. 28. 14. 22. 17. 22. 23.]
 [27. 10. 17. 10. 17. 28.  0. 12. 18. 26. 10. 12.]
 [13. 15. 27. 20. 19. 14. 12.  0. 27. 10. 19. 17.]
 [29. 12. 19. 21. 20. 22. 18. 27.  0. 20. 22. 16.]
 [11. 20. 24. 10. 14. 17. 26. 10. 10.  0. 25. 21.]
 [18. 19. 12. 14. 14. 22. 10. 19. 12. 25.  0. 21.]
 [20. 21. 17. 10. 17. 20. 22. 17. 16. 21. 21.  0.]]


In [32]:
print(data_txtfile2)

[[ 0. 18. 22. 19. 23. 20. 18. 19.  0. 20. 24. 35.]
 [21.  0. 20.  0. 19.  0. 22. 20. 18. 19.  0. 33.]
 [22. 23.  0.  0.  0. 17. 16. 24. 16. 18. 24.  0.]
 [17. 25.  0.  0. 21. 22. 20. 17. 15. 16. 24. 31.]
 [12. 19. 18. 19.  0. 21. 21. 23. 21.  0.  0. 21.]
 [20.  0.  0. 17. 21.  0. 20.  0. 19. 17. 22. 20.]
 [22. 24. 28. 16. 23.  0.  0. 19. 24. 25. 30. 19.]
 [23. 29. 20. 17. 24. 24. 23.  0. 19. 22. 30. 26.]
 [ 0. 28. 29. 21.  0. 24. 18. 18.  0.  0. 22. 23.]
 [31. 20.  0. 19. 23.  0. 20. 24. 19.  0. 20.  0.]
 [17.  0. 25. 23. 20. 18. 16. 19.  0. 39.  0. 34.]
 [ 0. 32. 24. 26.  0. 19.  0. 18. 37. 21. 20.  0.]]


In [33]:
model=ConcreteModel()

In [34]:
M = data_txtfile1.shape[1]-4
N=data_txtfile2.shape[0]-4

In [35]:
row_indices=range(M)
col_indices=range(N)
print(row_indices)
print(col_indices)


range(0, 8)
range(0, 8)


In [36]:
model.x=Var(row_indices,col_indices,domain=Binary)
model.y=Var(range(8),range(8),range(8),range(8),domain= NonNegativeReals)

In [37]:
model.constraint=ConstraintList()

In [38]:
model.objective = Objective(expr=sum(data_txtfile1[j][l]*data_txtfile2[i][k]*model.y[i,j,k,l]for i in row_indices for j in col_indices  for k in row_indices for l in col_indices),sense=minimize)

In [39]:
for i in range(0,8):
   for j in range(0,8):
      for k in range(0,8):
        for l in range(0,8):
          model.constraint.add(expr=model.y[i,j,k,l]==model.y[k,l,i,j])
          

In [40]:
for j in range(8):
  for k in range(8):
    for l in range(8):
      model.constraint.add(expr=sum(model.y[i,j,k,l] for i in range(8))==model.x[k,l])

In [41]:
for i in range(8):
  for k in range(8):
    for l in range(8):
      model.constraint.add(expr=sum(model.y[i,j,k,l] for j in range(8))==model.x[k,l])

In [42]:
for i in row_indices:
  model.constraint.add(expr = sum(model.x[i,j] for j in col_indices)==1)
for j in col_indices:
  model.constraint.add(expr=sum(model.x[i,j] for i in row_indices) == 1)


In [43]:
model.pprint()

Streaming output truncated to the last 5000 lines.
         139 :   0.0 :                                                                                        y[0,2,1,2] - y[1,2,0,2] :   0.0 :   True
         140 :   0.0 :                                                                                        y[0,2,1,3] - y[1,3,0,2] :   0.0 :   True
         141 :   0.0 :                                                                                        y[0,2,1,4] - y[1,4,0,2] :   0.0 :   True
         142 :   0.0 :                                                                                        y[0,2,1,5] - y[1,5,0,2] :   0.0 :   True
         143 :   0.0 :                                                                                        y[0,2,1,6] - y[1,6,0,2] :   0.0 :   True
         144 :   0.0 :                                                                                        y[0,2,1,7] - y[1,7,0,2] :   0.0 :   True
         145 :   0.0 :                     

In [44]:
!apt-get install -y -qq coinor-cbc

In [45]:
opt_cbc= SolverFactory('cbc',)
result = opt_cbc.solve(model)
print('solver statuas:',result.solver.status)
print('solver termination condition:',result.solver.termination_condition)
print('solver time taken:',result.solver.time)

solver statuas: ok
solver termination condition: optimal
solver time taken: 515.6015684604645


In [46]:
print("objective solution",model.objective())

objective solution 14889.0


In [47]:
#displaty solution
print('\nObjective=',model.objective())
print('\n decision variable')
for i  in row_indices:
  for j in col_indices:
    if model.x[i,j].value==1:
      print('Factory',i+1,'locate',j+1,':')



Objective= 14889.0

 decision variable
Factory 1 locate 2 :
Factory 2 locate 8 :
Factory 3 locate 7 :
Factory 4 locate 6 :
Factory 5 locate 1 :
Factory 6 locate 3 :
Factory 7 locate 5 :
Factory 8 locate 4 :


$\mathbf{2.}$ [R] If there are n factories and n locations, how many variables and constraints are there in
your model

$\mathbf{Solution:}$
Constraint: $n^4+2n^3+2n$

Binary Variable:$n^2$

Continous Variable:$n^4$

$\mathbf{9.}$ [R] Solve the problem and report which facility must be opened at each location.

$\mathbf{Solution:}$ 
Objective= 14889.0


Factory 1 locate 2 

Factory 2 locate 8 

Factory 3 locate 7 

Factory 4 locate 6 

Factory 5 locate 1 

Factory 6 locate 3 

Factory 7 locate 5 

Factory 8 locate 4 

$\mathbf{10.}$[R] Now change the integer variables in your model to continuous variables, and re-solve the
problem. Report the solution (only the non-zero values of the solution).


$\mathbf{Solution:}$

Objective= 11790.875010123458

 decision variable
* x[1,1] = 0.084983499
* x[1,2] = 0.33367696
* x[1,3] = 0.048316653
* x[1,4] = 0.10039981
* x[1,5] = 0.051444116
* x[1,6] = 0.25047975
* x[1,7] = 0.066562267
* x[1,8] = 0.064136939
* x[2,1] = 0.19073548
* x[2,2] = 0.026261001
* x[2,3] = 0.11864323
* x[2,4] = 0.1917243
* x[2,5] = 0.17399296
* x[2,6] = 0.022830959
* x[2,7] = 0.15180168
* x[2,8] = 0.1240104
* x[3,1] = 0.10388789
* x[3,3] = 0.17341885
* x[3,4] = 0.20991576
* x[3,5] = 0.15765049
* x[3,6] = 0.12777006
* x[3,7] = 0.12970289
* x[3,8] = 0.09765406
* x[4,1] = 0.07739104
* x[4,2] = 0.0017389244
* x[4,3] = 0.16883651
* x[4,4] = 0.041916945
* x[4,5] = 0.22827876
* x[4,6] = 0.13528148
* x[4,7] = 0.11864323
* x[4,8] = 0.22791312
* x[5,1] = 0.20537663
* x[5,2] = 0.14760491
* x[5,3] = 0.15701317
* x[5,4] = 0.043650923
* x[5,5] = 0.038651246
* x[5,6] = 0.0772711
* x[5,7] = 0.15673067
* x[5,8] = 0.17370134
* x[6,1] = 0.022830959
* x[6,3] = 0.19583209
* x[6,4] = 0.019085987
* x[6,5] = 0.26693
* x[6,6] = 0.32892201
* x[6,7] = 0.06176476
* x[6,8] = 0.1046342
* x[7,1] = 0.063140624
* x[7,2] = 0.31854682
* x[7,3] = 0.06176476
* x[7,4] = 0.26544384
* x[7,5] = 0.083052425
* x[7,6] = 0.023820054
* x[7,7] = 0.063140624
* x[7,8] = 0.12109086
* x[8,1] = 0.25165388
* x[8,2] = 0.17217138
* x[8,3] = 0.076174743
* x[8,4] = 0.12786244
* x[8,6] = 0.033624599
* x[8,7] = 0.25165388
* x[8,8] = 0.086859086

In [48]:
model.x.domain=Reals


In [49]:
model.x.setub(1)
model.x.setlb(0)

In [50]:
model.pprint()

Streaming output truncated to the last 5000 lines.
         139 :   0.0 :                                                                                        y[0,2,1,2] - y[1,2,0,2] :   0.0 :   True
         140 :   0.0 :                                                                                        y[0,2,1,3] - y[1,3,0,2] :   0.0 :   True
         141 :   0.0 :                                                                                        y[0,2,1,4] - y[1,4,0,2] :   0.0 :   True
         142 :   0.0 :                                                                                        y[0,2,1,5] - y[1,5,0,2] :   0.0 :   True
         143 :   0.0 :                                                                                        y[0,2,1,6] - y[1,6,0,2] :   0.0 :   True
         144 :   0.0 :                                                                                        y[0,2,1,7] - y[1,7,0,2] :   0.0 :   True
         145 :   0.0 :                     

In [51]:
opt_cbc= SolverFactory('cbc')

In [52]:
result = opt_cbc.solve(model)
print('solver statuas:',result.solver.status)
print('solver termination condition:',result.solver.termination_condition)

solver statuas: ok
solver termination condition: optimal


In [57]:
#displaty solution
print('\nObjective=',model.objective())
print('\n decision variable')
for i  in row_indices:
  for j in col_indices:
    if model.x[i,j].value !=0:
      print(f"* x[{i+1},{j+1}] =",model.x[i,j].value)



Objective= 11790.875010123458

 decision variable
* x[1,1] = 0.084983499
* x[1,2] = 0.33367696
* x[1,3] = 0.048316653
* x[1,4] = 0.10039981
* x[1,5] = 0.051444116
* x[1,6] = 0.25047975
* x[1,7] = 0.066562267
* x[1,8] = 0.064136939
* x[2,1] = 0.19073548
* x[2,2] = 0.026261001
* x[2,3] = 0.11864323
* x[2,4] = 0.1917243
* x[2,5] = 0.17399296
* x[2,6] = 0.022830959
* x[2,7] = 0.15180168
* x[2,8] = 0.1240104
* x[3,1] = 0.10388789
* x[3,3] = 0.17341885
* x[3,4] = 0.20991576
* x[3,5] = 0.15765049
* x[3,6] = 0.12777006
* x[3,7] = 0.12970289
* x[3,8] = 0.09765406
* x[4,1] = 0.07739104
* x[4,2] = 0.0017389244
* x[4,3] = 0.16883651
* x[4,4] = 0.041916945
* x[4,5] = 0.22827876
* x[4,6] = 0.13528148
* x[4,7] = 0.11864323
* x[4,8] = 0.22791312
* x[5,1] = 0.20537663
* x[5,2] = 0.14760491
* x[5,3] = 0.15701317
* x[5,4] = 0.043650923
* x[5,5] = 0.038651246
* x[5,6] = 0.0772711
* x[5,7] = 0.15673067
* x[5,8] = 0.17370134
* x[6,1] = 0.022830959
* x[6,3] = 0.19583209
* x[6,4] = 0.019085987
* x[6,5] = 0.2

$\mathbf{11.}$ [R] Are the optimal costs for both problems same? Are the values of the variables still
integer-valued? If yes, explain why.

$\mathbf{Solution:}$ No, the optimal costs for both problems are not same. because in real domain it can take real  feasible value so optimal solution in real is less than the integer domain solution.



